# Chapter 3: 3D Geometry and Data Structures

## Geometric Representations

This notebook covers:
- 3D primitive structures
- Mesh data structures
- Parametric curves and surfaces
- Implicit surfaces
- Procedural geometry

**Key References:** Marschner & Shirley Ch. 12, 15, Gambetta Ch. 4

---

## 1. 3D Primitives - Theory

### 1.1 Geometric Primitives

**3D primitives** are the fundamental building blocks of 3D graphics:

#### Point
A **point** in 3D space:
$$\mathbf{p} = (x, y, z) \in \mathbb{R}^3$$

#### Ray
A **ray** starts at origin $\mathbf{o}$ and extends in direction $\mathbf{d}$:
$$\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}, \quad t \geq 0$$

where:
- $\mathbf{o}$ is the ray origin
- $\mathbf{d}$ is the ray direction (usually normalized)
- $t$ is the parameter (distance along ray)

#### Line Segment
A **line segment** between points $\mathbf{a}$ and $\mathbf{b}$:
$$\mathbf{l}(t) = (1-t)\mathbf{a} + t\mathbf{b}, \quad t \in [0, 1]$$

#### Plane
A **plane** can be defined by:

**Point-normal form:**
$$(\mathbf{p} - \mathbf{p}_0) \cdot \mathbf{n} = 0$$

**Implicit form:**
$$ax + by + cz + d = 0$$

where $\mathbf{n} = (a, b, c)$ is the normal vector and $d$ is the distance from origin.

**General form:**
$$\mathbf{n} \cdot \mathbf{p} + d = 0$$

### 1.2 Ray-Plane Intersection

To find where ray $\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}$ intersects plane $\mathbf{n} \cdot \mathbf{p} + d_0 = 0$:

$$t = -\frac{\mathbf{n} \cdot \mathbf{o} + d_0}{\mathbf{n} \cdot \mathbf{d}}$$

**Conditions:**
- If $\mathbf{n} \cdot \mathbf{d} = 0$: ray parallel to plane (no intersection or infinite intersections)
- If $t < 0$: intersection behind ray origin
- If $t \geq 0$: intersection at $\mathbf{p} = \mathbf{o} + t\mathbf{d}$

### 1.3 Ray-Sphere Intersection

For sphere with center $\mathbf{c}$ and radius $r$:
$$\|\mathbf{p} - \mathbf{c}\|^2 = r^2$$

Substituting ray equation:
$$\|\mathbf{o} + t\mathbf{d} - \mathbf{c}\|^2 = r^2$$

Expanding:
$$(\mathbf{d} \cdot \mathbf{d})t^2 + 2\mathbf{d} \cdot (\mathbf{o} - \mathbf{c})t + (\mathbf{o} - \mathbf{c}) \cdot (\mathbf{o} - \mathbf{c}) - r^2 = 0$$

**Quadratic form:** $at^2 + bt + c = 0$

where:
- $a = \mathbf{d} \cdot \mathbf{d}$ (usually 1 if $\mathbf{d}$ normalized)
- $b = 2\mathbf{d} \cdot (\mathbf{o} - \mathbf{c})$
- $c = \|\mathbf{o} - \mathbf{c}\|^2 - r^2$

**Discriminant:** $\Delta = b^2 - 4ac$
- $\Delta < 0$: no intersection
- $\Delta = 0$: tangent (one intersection)
- $\Delta > 0$: two intersections

**Solutions:**
$$t = \frac{-b \pm \sqrt{\Delta}}{2a}$$

In [ ]:
# Example 1: Ray Intersection Testing with Multiple Objects
print("Example 1: Ray-Object Intersections\n")

# Create scene with multiple objects
objects = [
    Sphere(Vec3(0, 0, 5), 1.0),
    Sphere(Vec3(3, 0, 8), 0.8),
    Sphere(Vec3(-2, 1, 6), 0.6),
]

# Cast a ray
ray = Ray(Vec3(0, 0, 0), Vec3(0, 0, 1))

print(f"Casting ray: {ray}\n")

for idx, obj in enumerate(objects):
    result = obj.intersect_ray(ray)
    if result:
        t1, t2 = result
        print(f"Sphere {idx}: {obj}")
        print(f"  Intersections at t={t1:.3f}, t={t2:.3f}")
        print(f"  Hit points: {ray.at(t1)}, {ray.at(t2)}")
        print(f"  Normal at near hit: {obj.normal_at(ray.at(t1))}\n")
    else:
        print(f"Sphere {idx}: No intersection\n")

# Example 2: Build Complex Mesh from Primitives
print("\nExample 2: Complex Mesh Construction")

def create_combined_mesh():
    """Create a mesh combining multiple primitives"""
    combined = TriangleMesh()
    
    # Add a sphere
    sphere = create_sphere_mesh(radius=0.5, u_segments=15, v_segments=15)
    vertex_offset = len(combined.vertices)
    
    # Translate sphere up
    for v in sphere.vertices:
        combined.add_vertex(v + Vec3(0, 0, 1.5))
    
    for face in sphere.faces:
        i, j, k = face
        combined.add_face(i + vertex_offset, j + vertex_offset, k + vertex_offset)
    
    # Add a cylinder base
    cylinder = create_cylinder_mesh(radius=0.4, height=1.5, segments=20)
    vertex_offset = len(combined.vertices)
    
    for v in cylinder.vertices:
        combined.add_vertex(v)
    
    for face in cylinder.faces:
        i, j, k = face
        combined.add_face(i + vertex_offset, j + vertex_offset, k + vertex_offset)
    
    combined.compute_face_normals()
    combined.compute_vertex_normals()
    
    return combined

combined_mesh = create_combined_mesh()
print(f"Combined mesh: {combined_mesh}")
print(f"Bounds: {combined_mesh.bounds()}")
combined_mesh.visualize("Combined Mesh: Sphere on Cylinder")

# Example 3: Subdivision for Smoother Surfaces
print("\nExample 3: Mesh Analysis")

def analyze_mesh(mesh: TriangleMesh, name: str):
    """Analyze and print mesh statistics"""
    min_b, max_b = mesh.bounds()
    
    # Compute average face area
    total_area = 0
    for i in range(len(mesh.faces)):
        tri = mesh.get_triangle(i)
        total_area += tri.area()
    avg_area = total_area / len(mesh.faces) if mesh.faces else 0
    
    print(f"\n{name} Statistics:")
    print(f"  Vertices: {len(mesh.vertices)}")
    print(f"  Faces: {len(mesh.faces)}")
    print(f"  Euler characteristic: {mesh.euler_characteristic()}")
    print(f"  Bounding box: {min_b} to {max_b}")
    print(f"  Total surface area: {total_area:.3f}")
    print(f"  Average face area: {avg_area:.6f}")

# Analyze different resolution spheres
low_res_sphere = create_sphere_mesh(radius=1.0, u_segments=10, v_segments=10)
mid_res_sphere = create_sphere_mesh(radius=1.0, u_segments=20, v_segments=20)
high_res_sphere = create_sphere_mesh(radius=1.0, u_segments=40, v_segments=40)

analyze_mesh(low_res_sphere, "Low Resolution Sphere")
analyze_mesh(mid_res_sphere, "Medium Resolution Sphere")
analyze_mesh(high_res_sphere, "High Resolution Sphere")

# Example 4: Triangle Ray Intersection
print("\n\nExample 4: Ray-Triangle Intersection (Möller-Trumbore)")

# Create a triangle
tri = Triangle(
    Vec3(0, 0, 5),
    Vec3(2, 0, 5),
    Vec3(1, 2, 5)
)

# Test rays
test_rays = [
    Ray(Vec3(1, 0.5, 0), Vec3(0, 0, 1)),  # Should hit
    Ray(Vec3(0.5, 0.5, 0), Vec3(0, 0, 1)),  # Should hit
    Ray(Vec3(3, 3, 0), Vec3(0, 0, 1)),  # Should miss
    Ray(Vec3(1, 1, 0), Vec3(0, 1, 0)),  # Should miss (wrong direction)
]

print(f"Triangle: {tri}")
print(f"Triangle normal: {tri.normal}\n")

for idx, ray in enumerate(test_rays):
    result = tri.intersect_ray(ray)
    if result:
        t, u, v, w = result
        hit_point = ray.at(t)
        print(f"Ray {idx}: HIT at t={t:.3f}")
        print(f"  Hit point: {hit_point}")
        print(f"  Barycentric coords: ({w:.3f}, {u:.3f}, {v:.3f})\n")
    else:
        print(f"Ray {idx}: MISS\n")

print("\n✓ Chapter 3 Complete!")
print("\nIn this chapter, you learned:")
print("  • 3D geometric primitives (Ray, Plane, Sphere, Triangle)")
print("  • Ray-primitive intersection algorithms")
print("  • Triangle mesh data structures")
print("  • Vertex and face normal computation")
print("  • Parametric surfaces (Sphere, Cylinder, Torus)")
print("  • Euler characteristic and mesh topology")
print("\nNext Chapter: Viewing and Projection")

---

## 4. Practical Examples

Combining 3D primitives and meshes for complex scenes.

In [ ]:
def create_sphere_mesh(radius=1.0, u_segments=20, v_segments=20) -> TriangleMesh:
    """Create sphere mesh using parametric equations"""
    mesh = TriangleMesh()
    
    # Generate vertices
    for i in range(v_segments + 1):
        phi = math.pi * i / v_segments  # Latitude: 0 to π
        
        for j in range(u_segments + 1):
            theta = 2 * math.pi * j / u_segments  # Longitude: 0 to 2π
            
            # Spherical to Cartesian
            x = radius * math.sin(phi) * math.cos(theta)
            y = radius * math.sin(phi) * math.sin(theta)
            z = radius * math.cos(phi)
            
            mesh.add_vertex(Vec3(x, y, z))
    
    # Generate faces
    for i in range(v_segments):
        for j in range(u_segments):
            # Vertex indices
            v0 = i * (u_segments + 1) + j
            v1 = v0 + 1
            v2 = v0 + (u_segments + 1)
            v3 = v2 + 1
            
            # Two triangles per quad
            mesh.add_face(v0, v2, v1)
            mesh.add_face(v1, v2, v3)
    
    mesh.compute_face_normals()
    mesh.compute_vertex_normals()
    
    return mesh

def create_cylinder_mesh(radius=1.0, height=2.0, segments=20) -> TriangleMesh:
    """Create cylinder mesh"""
    mesh = TriangleMesh()
    
    # Generate vertices for cylinder body
    for i in range(2):  # Bottom and top
        h = -height/2 + i * height
        
        for j in range(segments + 1):
            theta = 2 * math.pi * j / segments
            x = radius * math.cos(theta)
            y = radius * math.sin(theta)
            z = h
            
            mesh.add_vertex(Vec3(x, y, z))
    
    # Generate faces for cylinder body
    for j in range(segments):
        v0 = j
        v1 = v0 + 1
        v2 = v0 + segments + 1
        v3 = v2 + 1
        
        mesh.add_face(v0, v2, v1)
        mesh.add_face(v1, v2, v3)
    
    # Add caps (top and bottom)
    # Bottom center
    bottom_center = mesh.add_vertex(Vec3(0, 0, -height/2))
    for j in range(segments):
        v0 = j
        v1 = j + 1
        mesh.add_face(bottom_center, v0, v1)
    
    # Top center
    top_center = mesh.add_vertex(Vec3(0, 0, height/2))
    offset = segments + 1
    for j in range(segments):
        v0 = offset + j
        v1 = offset + j + 1
        mesh.add_face(top_center, v1, v0)
    
    mesh.compute_face_normals()
    mesh.compute_vertex_normals()
    
    return mesh

def create_torus_mesh(major_radius=1.0, minor_radius=0.3, 
                      u_segments=30, v_segments=20) -> TriangleMesh:
    """Create torus mesh"""
    mesh = TriangleMesh()
    
    # Generate vertices
    for i in range(v_segments):
        phi = 2 * math.pi * i / v_segments  # Minor circle angle
        
        for j in range(u_segments):
            theta = 2 * math.pi * j / u_segments  # Major circle angle
            
            # Torus parametric equations
            x = (major_radius + minor_radius * math.cos(phi)) * math.cos(theta)
            y = (major_radius + minor_radius * math.cos(phi)) * math.sin(theta)
            z = minor_radius * math.sin(phi)
            
            mesh.add_vertex(Vec3(x, y, z))
    
    # Generate faces
    for i in range(v_segments):
        for j in range(u_segments):
            v0 = i * u_segments + j
            v1 = i * u_segments + (j + 1) % u_segments
            v2 = ((i + 1) % v_segments) * u_segments + j
            v3 = ((i + 1) % v_segments) * u_segments + (j + 1) % u_segments
            
            mesh.add_face(v0, v2, v1)
            mesh.add_face(v1, v2, v3)
    
    mesh.compute_face_normals()
    mesh.compute_vertex_normals()
    
    return mesh

# Create and visualize parametric surfaces
print("Creating parametric surfaces...\n")

# Sphere
sphere_mesh = create_sphere_mesh(radius=1.0, u_segments=30, v_segments=30)
print(f"Sphere: {sphere_mesh}")
print(f"Euler characteristic: {sphere_mesh.euler_characteristic()}")
sphere_mesh.visualize("Parametric Sphere")

# Cylinder
cylinder_mesh = create_cylinder_mesh(radius=0.8, height=2.0, segments=30)
print(f"\nCylinder: {cylinder_mesh}")
cylinder_mesh.visualize("Parametric Cylinder")

# Torus
torus_mesh = create_torus_mesh(major_radius=1.0, minor_radius=0.3, 
                               u_segments=40, v_segments=20)
print(f"\nTorus: {torus_mesh}")
print(f"Euler characteristic: {torus_mesh.euler_characteristic()} (genus=1, so should be 0)")
torus_mesh.visualize("Parametric Torus")

---

## 3. Parametric Surfaces - Implementation

---

## 3. Parametric Surfaces - Theory

### 3.1 Parametric Surface Definition

A **parametric surface** is defined by a function:
$$\mathbf{S}(u, v) = (x(u, v), y(u, v), z(u, v)), \quad (u, v) \in [u_0, u_1] \times [v_0, v_1]$$

Parameters $u$ and $v$ map to 3D points on the surface.

### 3.2 Surface Normal

Compute surface normal using partial derivatives:

**Tangent vectors:**
$$\mathbf{T}_u = \frac{\partial \mathbf{S}}{\partial u}, \quad \mathbf{T}_v = \frac{\partial \mathbf{S}}{\partial v}$$

**Normal vector:**
$$\mathbf{n}(u, v) = \frac{\mathbf{T}_u \times \mathbf{T}_v}{\|\mathbf{T}_u \times \mathbf{T}_v\|}$$

### 3.3 Common Parametric Surfaces

#### Sphere
$$\mathbf{S}(\theta, \phi) = \begin{pmatrix} r\sin\phi\cos\theta \\ r\sin\phi\sin\theta \\ r\cos\phi \end{pmatrix}$$

where $\theta \in [0, 2\pi]$ (longitude), $\phi \in [0, \pi]$ (latitude)

#### Cylinder
$$\mathbf{S}(\theta, h) = \begin{pmatrix} r\cos\theta \\ r\sin\theta \\ h \end{pmatrix}$$

where $\theta \in [0, 2\pi]$, $h \in [0, H]$

#### Torus
$$\mathbf{S}(\theta, \phi) = \begin{pmatrix} (R + r\cos\phi)\cos\theta \\ (R + r\cos\phi)\sin\theta \\ r\sin\phi \end{pmatrix}$$

where:
- $R$ = major radius (distance from origin to tube center)
- $r$ = minor radius (tube radius)
- $\theta, \phi \in [0, 2\pi]$

### 3.4 Bézier Surfaces

A **Bézier surface** is defined by control points $\mathbf{P}_{ij}$:

$$\mathbf{S}(u, v) = \sum_{i=0}^{m} \sum_{j=0}^{n} B_i^m(u) B_j^n(v) \mathbf{P}_{ij}$$

where $B_i^n(t) = \binom{n}{i}t^i(1-t)^{n-i}$ are Bernstein polynomials.

**Properties:**
- Interpolates corner control points
- Lies within convex hull of control points
- Smooth ($C^\infty$ continuous)

In [ ]:
class TriangleMesh:
    """Triangle mesh data structure"""
    
    def __init__(self):
        self.vertices = []  # List of Vec3
        self.faces = []     # List of (i, j, k) tuples (vertex indices)
        self.vertex_normals = []  # List of Vec3
        self.face_normals = []    # List of Vec3
    
    def add_vertex(self, v: Vec3) -> int:
        """Add vertex and return its index"""
        self.vertices.append(v)
        return len(self.vertices) - 1
    
    def add_face(self, i: int, j: int, k: int):
        """Add triangular face (vertex indices)"""
        self.faces.append((i, j, k))
    
    def compute_face_normals(self):
        """Compute normal for each face"""
        self.face_normals = []
        for i, j, k in self.faces:
            v0, v1, v2 = self.vertices[i], self.vertices[j], self.vertices[k]
            edge1 = v1 - v0
            edge2 = v2 - v0
            normal = edge1.cross(edge2).normalize()
            self.face_normals.append(normal)
    
    def compute_vertex_normals(self):
        """Compute smooth normals at each vertex (average of adjacent faces)"""
        if not self.face_normals:
            self.compute_face_normals()
        
        # Initialize vertex normals to zero
        self.vertex_normals = [Vec3(0, 0, 0) for _ in self.vertices]
        
        # Accumulate face normals
        for face_idx, (i, j, k) in enumerate(self.faces):
            normal = self.face_normals[face_idx]
            self.vertex_normals[i] = self.vertex_normals[i] + normal
            self.vertex_normals[j] = self.vertex_normals[j] + normal
            self.vertex_normals[k] = self.vertex_normals[k] + normal
        
        # Normalize
        self.vertex_normals = [n.normalize() for n in self.vertex_normals]
    
    def get_triangle(self, face_idx: int) -> Triangle:
        """Get Triangle object for a face"""
        i, j, k = self.faces[face_idx]
        return Triangle(self.vertices[i], self.vertices[j], self.vertices[k])
    
    def euler_characteristic(self) -> int:
        """Compute V - E + F"""
        V = len(self.vertices)
        F = len(self.faces)
        
        # Count unique edges
        edges = set()
        for i, j, k in self.faces:
            edges.add(tuple(sorted([i, j])))
            edges.add(tuple(sorted([j, k])))
            edges.add(tuple(sorted([k, i])))
        E = len(edges)
        
        return V - E + F
    
    def bounds(self) -> Tuple[Vec3, Vec3]:
        """Compute axis-aligned bounding box"""
        if not self.vertices:
            return (Vec3(0, 0, 0), Vec3(0, 0, 0))
        
        min_x = min(v.x for v in self.vertices)
        min_y = min(v.y for v in self.vertices)
        min_z = min(v.z for v in self.vertices)
        max_x = max(v.x for v in self.vertices)
        max_y = max(v.y for v in self.vertices)
        max_z = max(v.z for v in self.vertices)
        
        return (Vec3(min_x, min_y, min_z), Vec3(max_x, max_y, max_z))
    
    def visualize(self, title="Triangle Mesh", show_normals=False):
        """Visualize mesh using matplotlib"""
        if not self.vertices or not self.faces:
            print("Empty mesh!")
            return
        
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, projection='3d')
        
        # Convert faces to vertex arrays for plotting
        verts = []
        for i, j, k in self.faces:
            v0 = self.vertices[i].to_array()
            v1 = self.vertices[j].to_array()
            v2 = self.vertices[k].to_array()
            verts.append([v0, v1, v2])
        
        # Create 3D polygon collection
        poly = Poly3DCollection(verts, alpha=0.7, facecolor='cyan', edgecolor='darkblue')
        ax.add_collection3d(poly)
        
        # Plot vertex normals if requested
        if show_normals and self.vertex_normals:
            for v, n in zip(self.vertices, self.vertex_normals):
                start = v.to_array()
                end = (v + n * 0.2).to_array()
                ax.plot([start[0], end[0]], [start[1], end[1]], [start[2], end[2]], 
                       'r-', linewidth=1)
        
        # Set axis limits
        min_b, max_b = self.bounds()
        ax.set_xlim([min_b.x, max_b.x])
        ax.set_ylim([min_b.y, max_b.y])
        ax.set_zlim([min_b.z, max_b.z])
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(title)
        
        plt.tight_layout()
        plt.show()
    
    def __repr__(self):
        return f"TriangleMesh(vertices={len(self.vertices)}, faces={len(self.faces)})"

# Create a simple cube mesh
def create_cube(size=1.0) -> TriangleMesh:
    """Create a cube mesh"""
    mesh = TriangleMesh()
    s = size / 2
    
    # 8 vertices of a cube
    vertices = [
        Vec3(-s, -s, -s),  # 0
        Vec3( s, -s, -s),  # 1
        Vec3( s,  s, -s),  # 2
        Vec3(-s,  s, -s),  # 3
        Vec3(-s, -s,  s),  # 4
        Vec3( s, -s,  s),  # 5
        Vec3( s,  s,  s),  # 6
        Vec3(-s,  s,  s),  # 7
    ]
    
    for v in vertices:
        mesh.add_vertex(v)
    
    # 12 triangular faces (2 per cube face)
    faces = [
        # Front
        (0, 1, 2), (0, 2, 3),
        # Back
        (5, 4, 7), (5, 7, 6),
        # Left
        (4, 0, 3), (4, 3, 7),
        # Right
        (1, 5, 6), (1, 6, 2),
        # Top
        (3, 2, 6), (3, 6, 7),
        # Bottom
        (4, 5, 1), (4, 1, 0),
    ]
    
    for face in faces:
        mesh.add_face(*face)
    
    mesh.compute_face_normals()
    mesh.compute_vertex_normals()
    
    return mesh

# Test mesh
cube = create_cube(2.0)
print(f"Cube mesh: {cube}")
print(f"Euler characteristic: {cube.euler_characteristic()} (should be 2 for sphere topology)")
print(f"Bounds: {cube.bounds()}")
print(f"Number of vertex normals: {len(cube.vertex_normals)}")
print(f"First vertex normal: {cube.vertex_normals[0]}")

# Visualize
cube.visualize("Cube Mesh", show_normals=True)

---

## 2. Mesh Data Structures - Implementation

---

## 2. Mesh Data Structures - Theory

### 2.1 Triangle Mesh

A **triangle mesh** represents a 3D surface using triangular faces.

**Components:**
- **Vertices:** Set of 3D points $V = \{\mathbf{v}_0, \mathbf{v}_1, \ldots, \mathbf{v}_{n-1}\}$
- **Faces:** Set of triangles $F = \{(i_0, j_0, k_0), (i_1, j_1, k_1), \ldots\}$
  - Each face is a tuple of three vertex indices

### 2.2 Mesh Topology

**Manifold mesh:** Every edge is shared by exactly 1 or 2 triangles

**Non-manifold conditions:**
- Edge shared by > 2 triangles
- Vertices not connected to form a continuous surface

### 2.3 Vertex Normals

**Face normal** (flat shading):
$$\mathbf{n}_{\text{face}} = \frac{(\mathbf{v}_1 - \mathbf{v}_0) \times (\mathbf{v}_2 - \mathbf{v}_0)}{\|(\mathbf{v}_1 - \mathbf{v}_0) \times (\mathbf{v}_2 - \mathbf{v}_0)\|}$$

**Vertex normal** (smooth shading) - average of adjacent face normals:
$$\mathbf{n}_{\text{vertex}} = \frac{\sum_{f \in \text{adjacent faces}} \mathbf{n}_f}{\|\sum_{f \in \text{adjacent faces}} \mathbf{n}_f\|}$$

**Weighted vertex normal:** Weight by face area or angle
$$\mathbf{n}_{\text{vertex}} = \frac{\sum_{f} A_f \mathbf{n}_f}{\|\sum_{f} A_f \mathbf{n}_f\|}$$

### 2.4 Half-Edge Data Structure

Efficient representation for mesh traversal:

**Half-edge** $e$:
- `vertex`: vertex at the end of this half-edge
- `face`: face to the left of this half-edge  
- `twin`: opposite half-edge
- `next`: next half-edge around the face
- `prev`: previous half-edge around the face

**Advantages:**
- $O(1)$ access to adjacency information
- Easy mesh traversal and modification

### 2.5 Mesh Properties

**Euler characteristic** (for closed manifold mesh):
$$V - E + F = 2(1 - g)$$

where:
- $V$ = number of vertices
- $E$ = number of edges
- $F$ = number of faces
- $g$ = genus (number of "holes")

For a sphere: $g = 0$, so $V - E + F = 2$

In [ ]:
class Ray:
    """Ray primitive: origin + t * direction"""
    
    def __init__(self, origin: Vec3, direction: Vec3):
        self.origin = origin
        self.direction = direction.normalize()
    
    def at(self, t: float) -> Vec3:
        """Get point along ray at parameter t"""
        return self.origin + self.direction * t
    
    def __repr__(self):
        return f"Ray(origin={self.origin}, direction={self.direction})"

class Plane:
    """Plane primitive: defined by point and normal"""
    
    def __init__(self, point: Vec3, normal: Vec3):
        self.point = point
        self.normal = normal.normalize()
        # Compute d for implicit form: n·p + d = 0
        self.d = -self.normal.dot(point)
    
    def distance_to_point(self, p: Vec3) -> float:
        """Signed distance from point to plane"""
        return self.normal.dot(p) + self.d
    
    def intersect_ray(self, ray: Ray) -> Optional[float]:
        """
        Find ray-plane intersection
        Returns t value or None if no intersection
        """
        denom = self.normal.dot(ray.direction)
        
        # Check if ray is parallel to plane
        if abs(denom) < 1e-6:
            return None
        
        t = -(self.normal.dot(ray.origin) + self.d) / denom
        
        # Only return positive t (intersection in front of ray)
        return t if t >= 0 else None
    
    def __repr__(self):
        return f"Plane(point={self.point}, normal={self.normal})"

class Sphere:
    """Sphere primitive: center and radius"""
    
    def __init__(self, center: Vec3, radius: float):
        self.center = center
        self.radius = radius
    
    def intersect_ray(self, ray: Ray) -> Optional[Tuple[float, float]]:
        """
        Find ray-sphere intersection
        Returns (t_near, t_far) or None if no intersection
        """
        oc = ray.origin - self.center
        
        # Quadratic equation coefficients
        a = ray.direction.dot(ray.direction)
        b = 2.0 * oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        
        # Discriminant
        discriminant = b * b - 4 * a * c
        
        if discriminant < 0:
            return None  # No intersection
        
        sqrt_d = math.sqrt(discriminant)
        t1 = (-b - sqrt_d) / (2 * a)
        t2 = (-b + sqrt_d) / (2 * a)
        
        # Return both intersection points
        return (t1, t2)
    
    def normal_at(self, point: Vec3) -> Vec3:
        """Get surface normal at point on sphere"""
        return (point - self.center).normalize()
    
    def __repr__(self):
        return f"Sphere(center={self.center}, radius={self.radius})"

class Triangle:
    """Triangle primitive: three vertices"""
    
    def __init__(self, v0: Vec3, v1: Vec3, v2: Vec3):
        self.v0 = v0
        self.v1 = v1
        self.v2 = v2
        # Compute normal using cross product
        edge1 = v1 - v0
        edge2 = v2 - v0
        self.normal = edge1.cross(edge2).normalize()
    
    def intersect_ray(self, ray: Ray) -> Optional[Tuple[float, float, float, float]]:
        """
        Möller-Trumbore ray-triangle intersection
        Returns (t, u, v, w) barycentric coords or None
        """
        edge1 = self.v1 - self.v0
        edge2 = self.v2 - self.v0
        
        h = ray.direction.cross(edge2)
        a = edge1.dot(h)
        
        # Check if ray is parallel to triangle
        if abs(a) < 1e-6:
            return None
        
        f = 1.0 / a
        s = ray.origin - self.v0
        u = f * s.dot(h)
        
        if u < 0.0 or u > 1.0:
            return None
        
        q = s.cross(edge1)
        v = f * ray.direction.dot(q)
        
        if v < 0.0 or u + v > 1.0:
            return None
        
        t = f * edge2.dot(q)
        
        if t > 1e-6:  # Ray intersection
            w = 1.0 - u - v
            return (t, u, v, w)
        
        return None
    
    def area(self) -> float:
        """Compute triangle area"""
        edge1 = self.v1 - self.v0
        edge2 = self.v2 - self.v0
        return 0.5 * edge1.cross(edge2).length()
    
    def __repr__(self):
        return f"Triangle({self.v0}, {self.v1}, {self.v2})"

# Test primitives
print("Testing 3D Primitives:\n")

# Ray
ray = Ray(Vec3(0, 0, 0), Vec3(1, 0, 0))
print(f"Ray: {ray}")
print(f"Point at t=5: {ray.at(5)}\n")

# Plane
plane = Plane(Vec3(5, 0, 0), Vec3(1, 0, 0))
print(f"Plane: {plane}")
t_plane = plane.intersect_ray(ray)
if t_plane:
    print(f"Ray-plane intersection at t={t_plane}, point={ray.at(t_plane)}\n")

# Sphere
sphere = Sphere(Vec3(10, 0, 0), 2.0)
print(f"Sphere: {sphere}")
result = sphere.intersect_ray(ray)
if result:
    t1, t2 = result
    print(f"Ray-sphere intersections at t={t1:.3f} and t={t2:.3f}")
    print(f"  Near point: {ray.at(t1)}")
    print(f"  Far point: {ray.at(t2)}\n")

# Triangle
tri = Triangle(Vec3(0, 0, 0), Vec3(1, 0, 0), Vec3(0, 1, 0))
print(f"Triangle: {tri}")
print(f"Triangle normal: {tri.normal}")
print(f"Triangle area: {tri.area():.3f}")

---

## 1. 3D Primitives - Implementation

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import math
from typing import Tuple, Optional, List